In [ ]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.model.supervised import Supervised
from nlptools.utils import Config

cfg = Config("../config/cornell_transformer.yml")
model = Supervised.build(cfg)

In [2]:
dialog = iter(model.generator).next()
dialog.to("cuda:0")
response = dialog["response_cornell"].data
response_mask = dialog["response_mask_cornell"].data
utterance = dialog["utterance"].data
utterance_mask = dialog["utterance_mask"].data
sentiment = dialog["sentiment"].data
print(utterance, utterance.shape)
print(utterance_mask, utterance_mask.shape)
print(response, response.shape)
print(response_mask, response_mask.shape)
print(sentiment[:10, :], sentiment.shape)

tensor([[ 101, 1045, 2113,  ...,    0,    0,    0],
        [ 101, 2296, 2051,  ..., 2572, 2770,  102],
        [ 101, 2009, 2001,  ...,    0,    0,    0],
        ...,
        [ 101, 2002, 2056,  ...,    0,    0,    0],
        [ 101, 2115, 4113,  ..., 1012,  102,    0],
        [ 101, 2074, 2115,  ...,    0,    0,    0]], device='cuda:0') torch.Size([63, 20])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0') torch.Size([63, 20])
tensor([[ 101, 2129, 1040,  ...,    0,    0,    0],
        [ 101, 1045, 2245,  ...,    0,    0,    0],
        [ 101, 2339, 2079,  ..., 2514, 2488,  102],
        ...,
        [ 101, 2002, 2056,  ...,    0,    0,    0],
        [ 101, 2002, 2003,  ...,    0,    0,    0],
        [ 101, 2029, 2003,  ...,    0,    0,    0]], device='cuda:0') torch.Size([63, 20])
tensor([[1, 1, 1

In [3]:
encoder_out, encoder_hidden = model.skill.model.dialog_embedding(utterance, utterance_mask, sentiment)
prev_output = response.data[:, :-1]
target_output = response.data[:, 1:]
print(encoder_out.shape)
print(prev_output.shape)
print(target_output.shape)

torch.Size([63, 20, 768])
torch.Size([63, 19])
torch.Size([63, 19])


In [4]:
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
target_output.shape

torch.Size([1197])

In [6]:
#import nlptools.zoo.encoders.transformer
import nlptools.zoo.encoders.gru
import nlptools.zoo.encoders.attention
import importlib
#importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(nlptools.zoo.encoders.gru)
importlib.reload(nlptools.zoo.encoders.attention)
#from nlptools.zoo.encoders.transformer import TransformerDecoder, TransformerEncoder
from nlptools.zoo.encoders.gru import GRUDecoder

embedding = model.skill.model.encoder.embedding
decoder_hidden_layers = 1
decoder_attention_heads = 3
decoder_hidden_size = 2048
dropout = 0
# decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout=dropout)
decoder = GRUDecoder(embedding, decoder_hidden_layers, decoder_hidden_size, dropout=dropout)
decoder = decoder.to("cuda:0")
decoder

GRUDecoder(
  (word_embedding): Embedding(30522, 768)
  (gru): GRU(768, 1024, batch_first=True)
  (attn): Linear(in_features=1792, out_features=20, bias=True)
  (attn_combine): Linear(in_features=1536, out_features=768, bias=True)
  (dropout): Dropout(p=0)
  (hidden_proj): Linear(in_features=768, out_features=1024, bias=True)
  (intermediate_linear): Linear(in_features=1024, out_features=768, bias=True)
  (output_linear): Linear(in_features=768, out_features=30522, bias=False)
)

In [5]:
output = model.skill.model.decoder(prev_output, encoder_out, utterance_mask, encoder_hidden)
output_probs = model.skill.model.logsoftmax(output)
output_probs_expand = output_probs.contiguous().view(-1, output_probs.size(2))
loss = model.skill.model.loss_function(output_probs_expand, target_output)
loss.item()

19.192840576171875